In [2]:
import requests
import json
import datetime
import pandas as pd
import re
import os
import random
import urllib3
import time

In [3]:
from concurrent.futures import ThreadPoolExecutor, as_completed, wait
from datetime import datetime
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from pandas import json_normalize

headers = {
      'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
}
urllib3.disable_warnings()

In [1]:
path = '../../data/tw/raw/'

# 1. Get links 

In [7]:
url = 'https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=7'
res = requests.get(url,headers = headers)
soup = BeautifulSoup(res.text)
news_html_list = soup.find('ul', attrs={'class':'boxTitle'}).find_all('li')
news_html_list


[<li>
 <a class="ph" data-desc="P:0:全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋" href="https://news.ltn.com.tw/news/politics/breakingnews/3042246" title="全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋">
 <img alt="全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋" class="lazy_imgs" data-src="https://img.ltn.com.tw/Upload/news/300/2020/01/16/3042246_1_1.jpg" onerror="setDefault(this)" src="https://news.ltn.com.tw/assets/images/default.png" title="全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋"/>
 </a>
 <div class="cont" href="https://news.ltn.com.tw/news/politics/breakingnews/3042246">
 <a class="tit" data-desc="T:0:全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋" href="https://news.ltn.com.tw/news/politics/breakingnews/3042246" title="全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋">全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋</a>
 <i class="immtag chan" href="https://news.ltn.com.tw/news/politics/breakingnews/3042246">政治</i>
 <span class="time">2020/01/16</span>
 <a class="http" data-desc="T:0:全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋" href="https://news.ltn.com.tw/news/politics/breakingnews/3042246" title="全國得票率第一！ 陳亭妃勝選不發雞排 週六贈2千福袋">h

In [ ]:
news_html = news_html_list[0]
NewsMeta(news_html)

## a. define all functions

In [ ]:
def GetLinks1page(keyword, page):
    news_meta_list = []
    url = f'https://search.ltn.com.tw/list?keyword={keyword}&start_time=20190901&end_time=20200118&sort=date&type=all&page={page}'
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text)
    
    print(url)
    
    # if theres no result in this page: return none
    if soup.find('ul', attrs={'class':'boxTitle'}) is None:
        return "noresults"
    
    news_html_list = soup.find('ul', attrs={'class':'boxTitle'}).find_all('li')
    
    for news_html in news_html_list:
        news_meta_list.append(NewsMeta(news_html)) 
    
    news_meta_list = list(filter(lambda x: x is not None, news_meta_list))

    return news_meta_list


In [10]:
def random_sleep(page, next_sleep):
    next_sleep = next_sleep
    if page % 7 == 0:
            time.sleep(random.uniform(17, 23))
    elif page == next_sleep:
        time.sleep(random.uniform(3, 7))
        next_sleep =  random.uniform(1, 5) + page
        
    return next_sleep


In [11]:
def GetAllLinks(keyword):
    news = []
    next_sleep = 3
    for page in tqdm(range(300)):
        # prevent been block 
        next_sleep = random_sleep(page, next_sleep)
        
        tmp_news = GetLinks1page(keyword, page)
        
        if tmp_news =="noresults": break
            
        news += tmp_news
    return news

## a. get links acc to keywords

In [12]:
ltn_tsai = GetAllLinks('蔡英文')
len(ltn_tsai)

  0%|          | 0/300 [00:00<?, ?it/s]

https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=0
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=1
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=2
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=3
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=4
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=5
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=6
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=7
https://search.ltn.com.tw/list?keyword=蔡英文&start_time=20190901&end_time=20200118&sort=date&type=all&page=8
https://search.ltn.com.tw/list?keywor

4845

In [13]:
# convert JSON list to Pandas dataframe
ltn_tsai_df = pd.DataFrame.from_records(ltn_tsai)
ltn_tsai_df['keyword'] = 'tsai'

# define the CSV file path
csv_file = "ltn_tsai.csv"

# save dataframe to CSV file
ltn_tsai_df.to_csv(path + csv_file, index=False)

In [23]:
ltn_han = GetAllLinks('韓國瑜')
print(len(ltn_han))

ltn_han_df = pd.DataFrame.from_records(ltn_han)
ltn_han_df['keyword'] = 'han'
csv_file = "ltn_han.csv"
ltn_han_df.to_csv(path + csv_file, index=False)

  0%|          | 0/300 [00:00<?, ?it/s]

https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=0
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=1
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=2
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=3
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=4
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=5
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=6
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=7
https://search.ltn.com.tw/list?keyword=韓國瑜&start_time=20190901&end_time=20200118&sort=date&type=all&page=8
https://search.ltn.com.tw/list?keywor

In [15]:
ltn_dpp = GetAllLinks('民進黨')
print(len(ltn_dpp))

ltn_dpp_df = pd.DataFrame.from_records(ltn_dpp)
ltn_dpp_df['keyword'] = 'dpp' 
csv_file = "ltn_dpp.csv"
ltn_dpp_df.to_csv(path + csv_file, index=False)

  0%|          | 0/300 [00:00<?, ?it/s]

https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=0
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=1
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=2
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=3
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=4
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=5
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=6
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=7
https://search.ltn.com.tw/list?keyword=民進黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=8
https://search.ltn.com.tw/list?keywor

In [17]:
ltn_kmt = GetAllLinks('國民黨')
len(ltn_kmt)

ltn_kmt_df = pd.DataFrame.from_records(ltn_kmt)
ltn_kmt_df['keyword'] = 'kmt' 

csv_file = "ltn_kmt.csv"
ltn_kmt_df.to_csv(path + csv_file, index=False)

  0%|          | 0/300 [00:00<?, ?it/s]

https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=0
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=1
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=2
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=3
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=4
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=5
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=6
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=7
https://search.ltn.com.tw/list?keyword=國民黨&start_time=20190901&end_time=20200118&sort=date&type=all&page=8
https://search.ltn.com.tw/list?keywor

## c. combine data

In [8]:
ltn_tsai_df = pd.read_csv(path + "ltn_tsai.csv", index_col=False)

ltn_han_df = pd.read_csv(path + "ltn_han.csv", index_col=False)

ltn_kmt_df = pd.read_csv(path + "ltn_kmt.csv", index_col=False)

ltn_dpp_df = pd.read_csv(path + "ltn_dpp.csv", index_col=False)



In [9]:
# preprocess of the cti data


#concat news metadata from all 4 keywords
ltn_df = pd.concat([ltn_tsai_df, ltn_han_df, ltn_kmt_df, ltn_dpp_df], axis=0)
ltn_df.head(3)

,url,id,title,release_at,keyword
0,https://news.ltn.com.tw/news/politics/breaking...,3043882,轟中共光說不練！ 劉家昌宣布籌組「中國台灣反共黨」,2020/01/17,tsai
1,https://news.ltn.com.tw/news/politics/breaking...,3043861,列出小英國防經濟成果 486先生讚：台灣真的沒這麼強過,2020/01/17,tsai
2,https://news.ltn.com.tw/news/politics/breaking...,3043824,獲台南鄉親投票力挺 賴清德幫小英轉達「定不負所託」,2020/01/17,tsai


In [10]:
# preprocess of the ltn data


#concat news metadata from all 4 keywords
ltn_df = pd.concat([ltn_tsai_df, ltn_han_df, ltn_kmt_df, ltn_dpp_df], axis=0)
print(ltn_df.shape)

# convert the keyword data to onehot
ltn_df = pd.concat([ltn_df, pd.get_dummies(ltn_df.keyword, prefix='keyword')], axis=1)
ltn_df.drop('keyword', axis=1, inplace=True)
print(ltn_df.shape)


ltn_df = ltn_df.set_index('id')
ltn_df[['keyword_dpp', 'keyword_han', 'keyword_kmt', 'keyword_tsai']] = ltn_df.groupby('id')['keyword_dpp', 'keyword_han', 'keyword_kmt', 'keyword_tsai'].sum()
ltn_df = ltn_df.drop_duplicates(keep='last')
ltn_df = ltn_df.reset_index()

print(ltn_df.shape)
ltn_df.head()


(19127, 5)
(19127, 8)
(10195, 8)


/var/folders/_7/rk2bxwhd5wv3rvl66tld1lvr0000gn/T/ipykernel_15503/3952921666.py:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  Itn_df[['keyword_dpp', 'keyword_han', 'keyword_kmt', 'keyword_tsai']] = Itn_df.groupby('id')['keyword_dpp', 'keyword_han', 'keyword_kmt', 'keyword_tsai'].sum()


,id,url,title,release_at,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai
0,3043716,https://news.ltn.com.tw/news/politics/breaking...,台中想吃雞排看這裡 2500份19、20日手刀排一波,2020/01/17,0,0,0,2
1,3043404,https://news.ltn.com.tw/news/politics/breaking...,英國駐台代表晉見蔡總統 恭賀台灣蓬勃發展的民主機制,2020/01/17,0,0,0,2
2,3043321,https://news.ltn.com.tw/news/politics/breaking...,與國外知名網紅聊女權、寵物 蔡總統：台灣重能力不看性別,2020/01/17,0,0,0,2
3,3043393,https://news.ltn.com.tw/news/politics/breaking...,酈英傑：美台盼締造共同未來 是「真朋友真進展」,2020/01/17,0,0,0,2
4,3043325,https://news.ltn.com.tw/news/politics/breaking...,台美瓜宏四方會談 吳釗燮：促進區域合作及發展,2020/01/17,0,0,0,2


In [11]:
ltn_df.to_csv(path + 'ltn_news.csv', index=False)


# 2. get all news

In [7]:

# ltn_contents_df = pd.read_csv(path + 'ltn_contents.csv')
# print(len(ltn_contents_df))

5416


In [8]:
ltn_df = pd.read_csv(path + 'ltn_news.csv')
print(ltn_df.shape)
ltn_df.id.nunique()

(10195, 8)


10195

In [9]:
def GetResponse(idx, url): 
    if int(idx) % 7 == 0:
        time.sleep(random.uniform(15, 25))
    elif int(idx) % 3 ==0:
        time.sleep(random.uniform(1, 5))
        
    try :
        res = requests.get(url)
        return {"id": idx, "res": res.text}
    except:
        return None

In [10]:
def GetContent(idx, res):
    try:

        soup = BeautifulSoup(res)
        content = " ".join([p.text for p in soup.find('div', attrs={'data-desc':'內容頁'}).find_all('p')[:-1]])
        
        return {"id": idx, "content": content}
    
    except: 
        return None
    

In [11]:
df = ltn_df[5500:].sample(n=1000) # Need to set up the scraper page/range manually
Inv = 8
TreadNum = 8

ltn_contents=[]
start_time = time.time()

df = df.set_index('id')

Len = len(df)
for start in tqdm(range(0, Len, Inv)):


    end = start+Inv if start+Inv <= Len else Len
    # start the thread pool
    with ThreadPoolExecutor(2) as executor:
        responses = []
        futures = [executor.submit(GetResponse, idx, row['url']) for idx, row in df[start: end].iterrows()]
        
        for future in as_completed(futures):
            responses.append(future.result())

    # submit tasks and collect futures
#     responses = [GetResponse( index, row['url']) for index, row in df[start: end].iterrows()]           
            
        responses = [item for item in responses if item is not None]
        tmp_df = pd.DataFrame(responses).set_index("id")
        tmp_df = tmp_df.mask(tmp_df.eq('None')).dropna()
        if len(tmp_df) <=2: 
            print(start, end)
            break
        print(f"{len(tmp_df)} news got requests")


    with ThreadPoolExecutor(TreadNum) as executor:
        tmp_cnts = []
        futures = [executor.submit(GetContent, idx, row.res) for idx, row in tmp_df.iterrows()]

        for future in as_completed(futures):
            tmp_cnts.append(future.result())

        if tmp_cnts.count(None) > 0.5 * len(tmp_cnts):
            print(start, end)
            break
        else:  
            tmp_cnts = [item for item in tmp_cnts if item is not None]
            ltn_contents+=tmp_cnts

    print(f"{len(ltn_contents)} out of {10195} got contents\n")

end_time = time.time() 
print(end_time - start_time)



  0%|          | 0/125 [00:00<?, ?it/s]

8 news got requests
8 out of 10195 got contents

8 news got requests
16 out of 10195 got contents

8 news got requests
24 out of 10195 got contents

8 news got requests
32 out of 10195 got contents

8 news got requests
40 out of 10195 got contents

8 news got requests
48 out of 10195 got contents

8 news got requests
56 out of 10195 got contents

8 news got requests
64 out of 10195 got contents

8 news got requests
72 out of 10195 got contents

8 news got requests
80 out of 10195 got contents

8 news got requests
88 out of 10195 got contents

8 news got requests
96 out of 10195 got contents

8 news got requests
104 out of 10195 got contents

8 news got requests
112 out of 10195 got contents

8 news got requests
120 out of 10195 got contents

8 news got requests
128 out of 10195 got contents

8 news got requests
136 out of 10195 got contents

8 news got requests
144 out of 10195 got contents

8 news got requests
152 out of 10195 got contents

8 news got requests
160 out of 10195 got con

In [12]:
print(len(ltn_contents))
ltn_contents_df = pd.concat([ltn_contents_df, pd.DataFrame(ltn_contents)], axis=0).reset_index()
ltn_contents_df.to_csv(path + 'ltn_contents.csv', index=False)
print(len(ltn_contents_df))
ltn_contents_df[-5:]


1000
6416


,index,id,content
6411,995,3042889,蔡英文2018年選舉失利後辭去民進黨主席。（資料照） 〔即時新聞／綜合報導〕國民黨在2020...
6412,996,3007570,香港浸會大學學生會會長方仲賢（中）與香港大專學界國際事務代表團發言人張崑陽（右）、代表呂天忻...
6413,997,3036342,一邊一國行動黨候選人李婉鈺在選前最後一天舉辦「選前之午」，大搞創意，吸引選民目光，憶及新北市...
6414,998,3021108,關鍵時刻堅守民主！陳菊（左）呼籲新竹人集中選票支持鄭宏輝（中）進軍國會！（記者蔡彰盛攝） 〔...
6415,999,3019682,中華泛藍協會、反滲透法危機處理小組25日於立法院舉行「反滲透法的爆炸力」，文化大學副教授林忠...


In [13]:
# ltn_contents_df = ltn_contents_df.drop(columns=['level_0', 'index'])
ltn_contents_df.to_csv(path + 'ltn_contents.csv', index=False)

## b. combine contents and other meta data

In [33]:
ltn_contents_df

,id,content
0,3043404,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...
1,3043321,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...
2,3043393,美國在台協會與外交部17日舉辦「數位對話公共論壇：增進美台人民的關係」，美國在台協會處長酈英...
3,3043325,外交部長吳釗燮今在推特上推文表示，台美瓜宏舉行四方會談，推文照片並清楚可見四國國旗。（取自外...
4,3043291,行政院政務委員唐鳳將協助規劃成立數位發展部會，落實總統政見。（資料照，記者李欣芳攝） 〔記者...
...,...,...
5411,3008560,時代力量高鈺婷轟張善政24小時之內惹惱女性和一票理工人！（取自臉書） 〔記者蔡彰盛／竹市報導...
5412,3008662,《自由時報》11日公布民調顯示，韓國瑜目前民意支持度為15.47％。（資料照） 〔即時新聞／...
5413,1338927,國民黨副總統候選人張善政接受媒體訪問。（資料照） 記者鄒景雯／特稿 張善政先生的一句話：「因...
5414,3008485,中國東南衛視預告12月31日的跨年節目中有邱毅，李戡表示，邱若談及自己在台灣的選舉事宜，將違...


In [46]:
len(ltn_contents)

10192

In [73]:
# ltn_contents_df = ltn_contents_df.reset_index()
ltn_df = pd.merge(ltn_df, ltn_contents_df, on="id").drop(columns=['level_0', 'index'])
print(ltn_df.shape)
ltn_df.loc[ltn_df.isna().any(axis=1)]

(10192, 11)


,id,url,title,release_at,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,content_x,content_y,content
319,3023178,https://news.ltn.com.tw/news/politics/breaking...,慶祝建交20週年！陳建仁出訪帛琉 深化邦誼,2019/12/28,0,0,0,1,None,None,None
321,3022694,https://news.ltn.com.tw/news/politics/breaking...,宋楚瑜「自首」觸犯反滲透法 陸委會這樣說,2019/12/27,0,0,0,1,None,None,None
322,3022374,https://news.ltn.com.tw/news/politics/breaking...,稱投不下蔡英文 陳佩琪：公婆先生投誰與我無關,2019/12/27,0,0,0,1,None,None,None
323,3022329,https://news.ltn.com.tw/news/politics/breaking...,時代力量挺小英？徐永明：已多次表態蔡總統會連任,2019/12/27,0,0,0,1,None,None,None
324,3022010,https://news.ltn.com.tw/news/politics/breaking...,狠嗆波特王「痞子長那副拙相」 邱毅：讓你撩是自貶身價,2019/12/27,0,0,0,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
10187,3005383,https://news.ltn.com.tw/news/politics/breaking...,與蘇貞昌有心結？游錫堃陪蘇巧慧掃街打破傳聞,2019/12/11,1,0,0,0,None,None,None
10188,3005330,https://news.ltn.com.tw/news/politics/breaking...,杜絕賄選謠言 立委候選人李翁月娥豪氣簽500萬本票,2019/12/11,1,0,1,0,None,None,None
10189,3005347,https://news.ltn.com.tw/news/politics/breaking...,綠黨民調：蔡、韓差距近3成 近7成民眾對中共反感,2019/12/11,1,1,1,1,None,None,None
10190,3005360,https://news.ltn.com.tw/news/politics/breaking...,遭陳宜民推打還被指「怪阿姨」 北檢今傳女警作證,2019/12/11,1,0,1,0,None,None,None


In [53]:
# ltn_contents_df = pd.DataFrame(ltn_contents).set_index("id")
print(ltn_contents_df.shape)
ltn_df = ltn_df.merge(ltn_contents_df, on='id')
print(ltn_df.shape)
# ltn_df = ltn_df.mask(ltn_df.eq('None')).dropna()
# print(ltn_df.shape)

ltn_df.head(3)

(10192, 1)
(327, 12)


/var/folders/_7/rk2bxwhd5wv3rvl66tld1lvr0000gn/T/ipykernel_23076/3299012068.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'content_x'} in the result is deprecated and will raise a MergeError in a future version.
  Itn_df = Itn_df.merge(Itn_contents_df, on='id')


,id,url,title,release_at,keyword_dpp,keyword_han,keyword_kmt,keyword_tsai,content_x,content_y,content_x,content_y
0,3043716,https://news.ltn.com.tw/news/politics/breaking...,台中想吃雞排看這裡 2500份19、20日手刀排一波,2020/01/17,0,0,0,2,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...,館長兌現承諾發送雞排，其中2000份20日下午3點在陳柏惟烏日新興路481號服務處發放。（記...
1,3043404,https://news.ltn.com.tw/news/politics/breaking...,英國駐台代表晉見蔡總統 恭賀台灣蓬勃發展的民主機制,2020/01/17,0,0,0,2,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...,總統蔡英文（右）17日上午在總統府接見英國在台辦事處代表唐凱琳（Catherine Nett...
2,3043321,https://news.ltn.com.tw/news/politics/breaking...,與國外知名網紅聊女權、寵物 蔡總統：台灣重能力不看性別,2020/01/17,0,0,0,2,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...,總統蔡英文與百萬網紅塔米爾暢談各項議題。（中央社） 〔即時新聞／綜合報導〕總統蔡英文近期接受...


In [ ]:
ltn_df